In [1]:
from eval import *

In [2]:
newsgroups = fetch_20newsgroups(subset='all')
start = time.time()

checkpoint_dir = '/home/jcaip/workspace/quickthoughts/checkpoints/fluid_embeddings'
with open("{}/config.json".format(checkpoint_dir)) as fp:
    CONFIG = json.load(fp)

In [3]:
WV_MODEL = api.load(CONFIG['embedding'])
qt = QuickThoughts(WV_MODEL, hidden_size=CONFIG['hidden_size'])
trained_params = torch.load("{}/checkpoint_latest.pth".format(checkpoint_dir))
qt.load_state_dict(trained_params['state_dict'])
qt = qt.cuda()
qt.eval()
print("Restored successfully from {}".format(checkpoint_dir))

2019-07-12 08:22:43 INFO     loading projection weights from /home/jcaip/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2019-07-12 08:22:43 WARNING  this function is deprecated, use smart_open.open instead
2019-07-12 08:24:50 INFO     loaded (400000, 300) matrix from /home/jcaip/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
/home/jcaip/.conda/envs/ml/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Restored successfully from /home/jcaip/workspace/quickthoughts/checkpoints/fluid_embeddings


In [4]:
text, labels = newsgroups.data, newsgroups.target
test_batch_size=1000
size = len(labels)
print("Loaded dataset {} with total lines: {}".format("20 NEWS", size))

def make_batch(j):
    """Processes one test batch of the test datset"""
    stop_idx = min(size, j+test_batch_size)
    batch_text, batch_labels  = text[j:stop_idx], labels[j:stop_idx]
    data = list(map(lambda x: torch.LongTensor(prepare_sequence(x, WV_MODEL.vocab, no_zeros=True)), batch_text))
    for i in data:
        if len(i) == 0:
            print(i)
            input()
    packed = safe_pack_sequence(data).cuda()
    return qt(packed).cpu().detach().numpy()

feature_list = [make_batch(i) for i in range(0, size, test_batch_size)]
print("Processing {:5d} batches of size {:5d}".format(len(feature_list), test_batch_size))
features = np.concatenate(feature_list)
print("Test feature matrix of shape: {}".format(features.shape))

#shuffle
features, labels = shuffle(features, labels)

#X_train, X_test, y_train, y_test = train_test_split(features, labels)
#acc_t = fit_clf(X_train, y_train, X_test, y_test, 1)
#print("Trained on {:4d} examples - Test Accuracy: {:.2%}".format(len(X_train), acc_t))

Loaded dataset 20 NEWS with total lines: 18846
Processing    19 batches of size  1000
Test feature matrix of shape: (18846, 2000)


In [ ]:
from sklearn.cluster import KMeans, SpectralClustering

predicted_kmeans = KMeans(n_clusters=20, verbose=0).fit_predict(features[:5000])
predicted_spectral = SpectralClustering(n_clusters=20).fit_predict(features[:5000])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

feature_embedded = TSNE(n_components=2, verbose=1).fit_transform(features[:5000])

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[labels[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_spectral[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(20, 15))

fixed_xlim, fixed_ylim = (-85.6129455010451, 110.84618372125996), (-90.00594782812799, 80.691349744632)

for i in range(20):
    selected = feature_embedded[predicted_kmeans[:5000] == i]
    ax = axs[i//5, i%5]
    ax.scatter(selected[:, 0], selected[:, 1])
    ax.set_ylim(fixed_ylim)
    ax.set_xlim(fixed_xlim)
    
plt.show()

In [ ]:
for i in range(20):
    print("\nCategoty: {}\n".format(i))
    print("-------------------------------------------------------------------")
    for j in np.argsort(predicted_cluster[:, i])[:1]:
        print("Example :{}".format(j))
        print(text[j])